In [1]:
import networkx as nx
from scipy.io import mmread
import random
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily
import pandas as pd
from shapely import wkt
from tqdm import tqdm
from shapely.geometry import Point
import csv
import community
import numpy as np
import plotly.graph_objs as go
import random
import folium
import plotly


In [2]:
# We start by loading node labels and edges
node_labels = pd.read_csv('network-data/web-spam-detection.node_labels', header=None, names=['node', 'label'])
edges = pd.read_csv('network-data/web-spam-detection.edges', header=None, names=['source', 'target'])
# we then create the graph
G = nx.from_pandas_edgelist(edges, 'source', 'target') 
# adding node labels
node_labels_dict = dict(zip(node_labels['node'], node_labels['label']))
nx.set_node_attributes(G, node_labels_dict, 'label')

In [3]:
print("Number of nodes:", G.number_of_nodes()) 
print("Number of edges:", G.number_of_edges())

Number of nodes: 9072
Number of edges: 473854


In [4]:
# Get node positions using a layout algorithm (e.g., spring layout)
pos = nx.spring_layout(G)

In [5]:
from random import sample
import plotly.graph_objects as go

# Sample a subset of edges and nodes
sampled_edges = sample(list(G.edges()), 1000)  # Sample 1000 edges
sampled_nodes = sample(list(G.nodes()), 500)   # Sample 500 nodes

# Get node positions using a layout algorithm that provides 3D coordinates (e.g., spring layout)
pos = nx.spring_layout(G, dim=3)

# Create edges trace
edge_trace = go.Scatter3d(
    x=[],
    y=[],
    z=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Process sampled edges
for edge in tqdm(sampled_edges, desc="Processing edges"):
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)
    edge_trace['z'] += (z0, z1, None)

# Create nodes trace
node_trace = go.Scatter3d(
    x=[],
    y=[],
    z=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2
    )
)

# Process sampled nodes
for node in tqdm(sampled_nodes, desc="Processing nodes"):
    x, y, z = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)
    node_trace['z'] += (z,)
    node_trace['text'] += (f'Node: {node}<br>Label: {G.nodes[node]["label"]}<br>'
                           f'# of connections: {len(G.edges(node))}<br>',)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='<br>Interactive 3D Network Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[
                        dict(
                            text='Python script: <a href="https://plotly.com/">Plotly</a>',
                            showarrow=False,
                            xref='paper',
                            yref='paper',
                            x=0.005,
                            y=-0.002
                        )
                    ],
                    scene=dict(
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        zaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    )
                )
            )

# Display the plot
fig.show()


Processing nodes: 100%|██████████| 500/500 [00:00<00:00, 580.61it/s] 


In [ ]:
from random import sample
import networkx as nx
from pyvis.network import Network

# Sample a subset of edges and nodes
sampled_edges = sample(list(G.edges()), 2000)  # Sample 2000 edges
sampled_nodes = sample(list(G.nodes()), 1000)   # Sample 1000 nodes

# Prepare network data
nodes_data = []
for node in sampled_nodes:
    label = f"Node {node}"
    connections = len(G.edges(node))
    title = f"Node ID: {node}<br>Label: {label}</br># of connections: {connections}"
    nodes_data.append((str(node), {"label": label, "title": title}))

edges_data = [(str(edge[0]), str(edge[1]), {"title": "Edge"}) for edge in sampled_edges]

# Create Vis.js network object with cdn_resources='remote'
net = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes to the network
for node_data in nodes_data:
    net.add_node(node_data[0], **node_data[1])

# Add edges to the network
for edge_data in edges_data:
    source, target, _ = edge_data
    if source in net.nodes and target in net.nodes:  # Check if both nodes exist in the network
        net.add_edge(source, target, **edge_data[2])

# Define a JavaScript function for node click event
node_info_function = """
function showNodeInfo(data) {
    var node_id = data.nodes[0];
    var node_label = network.body.nodes[node_id].options.label;
    alert("Node ID: " + node_id + "\\nLabel: " + node_label);
}
"""

# Set the JavaScript function
net.set_options("""
var options = {
    "interaction": {
        "hover": true
    }
}
""")

# Display the Vis.js network
net.show_buttons(filter_=['physics'])
net.show("3d_network_3.html")


In [ ]:
# Additional imports
import community  # For community detection
import numpy as np
from networkx.algorithms.community import greedy_modularity_communities

# Community detection
communities = list(greedy_modularity_communities(G))

# Map nodes to communities
node_community_mapping = {}
for idx, community in enumerate(communities):
    for node in community:
        node_community_mapping[node] = idx

# Color nodes based on community
node_colors = [node_community_mapping[node] for node in G.nodes()]

# Update node trace with community colors
node_trace['marker']['color'] = node_colors

# Encode community detection as a facet
fig = plotly.tools.make_subplots(rows=1, cols=2, subplot_titles=("Original Network", "Community Detection"))

# Add original network to the first subplot
fig.append_trace(edge_trace, 1, 1)
fig.append_trace(node_trace, 1, 1)

# Add community detection to the second subplot
fig.append_trace(go.Scatter3d(x=node_trace['x'], y=node_trace['y'], z=node_trace['z'], 
                            mode='markers',
                            marker=dict(color=node_colors, size=5), 
                            hoverinfo='text', text=node_trace['text']), 1, 2)

# Update layout
fig['layout'].update(title='<br>Interactive 3D Network Graph with Community Detection',
                  titlefont_size=16,
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20, l=5, r=5, t=40),
                  annotations=[
                      dict(
                          text='Python script: <a href="https://plotly.com/">Plotly</a>',
                          showarrow=False,
                          xref='paper',
                          yref='paper',
                          x=0.005,
                          y=-0.002
                      )
                  ],
                  scene=dict(
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      zaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  ),
                  scene2=dict(
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      zaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  )
              )

# Display the plot
plotly.offline.iplot(fig)



### hover effect 

In [12]:
from random import sample
import networkx as nx
from pyvis.network import Network

# Sample a subset of edges and nodes
sampled_edges = sample(list(G.edges()), 5000)  # Sample 2000 edges
sampled_nodes = sample(list(G.nodes()), 3500)   # Sample 1000 nodes

# Prepare network data
nodes_data = []
for node in sampled_nodes:
    label = f"Node {node}"
    connections = len(G.edges(node))
    title = f"Node ID: {node}<br>Label: {label}</br># of connections: {connections}"
    nodes_data.append((str(node), {"label": label, "title": title}))

edges_data = [(str(edge[0]), str(edge[1]), {"title": "Edge"}) for edge in sampled_edges]

# Create Vis.js network object with cdn_resources='remote'
net = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes to the network with hover effect
for node_data in nodes_data:
    net.add_node(node_data[0], **node_data[1], hover="Node")

# Add edges to the network
for edge_data in edges_data:
    source, target, _ = edge_data
    if source in net.nodes and target in net.nodes:  # Check if both nodes exist in the network
        net.add_edge(source, target, **edge_data[2], hover="Edge")

# Display the Vis.js network
net.show("3d_network_2.html")


3d_network_2.html


In [15]:
from random import sample
import networkx as nx
from pyvis.network import Network

# Sample a subset of edges and nodes
sampled_edges = sample(list(G.edges()), 5000)  # Sample 5000 edges
sampled_nodes = sample(list(G.nodes()), 3500)   # Sample 3500 nodes

# Prepare network data
nodes_data = []
for node in sampled_nodes:
    label = f"Node {node}"
    connections = len(G.edges(node))
    title = f"Node ID: {node}<br>Label: {label}</br># of connections: {connections}"
    nodes_data.append((str(node), {"label": label, "title": title}))

edges_data = [(str(edge[0]), str(edge[1]), {"title": "Edge"}) for edge in sampled_edges]

# Create Vis.js network object with cdn_resources='remote'
net = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes to the network with hover effect
for node_data in nodes_data:
    net.add_node(node_data[0], **node_data[1], hover="Node")

# Add edges to the network
for edge_data in edges_data:
    source, target, _ = edge_data
    net.add_edge(source, target, **edge_data[2], hover="Edge")

# Function to filter nodes with edges
def filter_nodes_with_edges():
    nodes_with_edges = set()
    for source, target, _ in edges_data:
        nodes_with_edges.add(source)
        nodes_with_edges.add(target)
    for node in nodes_data:
        node_id = node[0]
        if node_id not in nodes_with_edges:
            net.hide_nodes([node_id])
        else:
            net.show_nodes([node_id])

# Add a button to filter nodes with edges
button_html = """
<button onclick="filterNodesWithEdges()">Filter Nodes with Edges</button>
<script type="text/javascript">
    function filterNodesWithEdges() {
        var kernel = Jupyter.notebook.kernel;
        kernel.execute('filter_nodes_with_edges()');
    }
</script>
"""
net.set_options('manipulation', True)  # Enable manipulation options
net.show_buttons(filter_=['nodes'])  # Show only node filter button

# Display the Vis.js network
net.show("3d_network_2.html")


AssertionError: non existent node '7892'